# **Apigee with Streaming server-sent events**

<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/apigee-samples/blob/main/llm-sse-security/llm_sse_security_v1.ipynb">
          <img src="https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/images/icon32.png?raw=true" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fapigee-samples%2Fmain%2Fllm-sse-security%2Fllm_sse_security_v1.ipynb">
          <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/apigee-samples/main/llm-sse-security/llm_sse_security_v1.ipynb">
          <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-sse-security/llm_sse_security_v1.ipynb">
          <img src="https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/images/github-mark.png?raw=true" width="30" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />
<br />

- This is a sample Apigee proxy to demonstrate Apigee with LLM workloads that streams server-sent events (SSE). SSE streaming reduces latency, and clients can receive response data as soon as it is generated by an LLM. This feature supports the use of AI agents that operate in real time environments, such as customer service bots or workflow orchestrators. For more info, check out this [page](https://cloud.google.com/apigee/docs/api-platform/develop/server-sent-events)
- In this sample, we will showcase how Apigee can extract the events from the target and send that to Model Armor to sanitize the model response. For information about Model Armor, see [Model Armor overview](https://cloud.google.com/security-command-center/docs/model-armor-overview)

![architecture](./images/arch.png)

# Benefits of Security with Apigee:

* Detect and block adversarial prompts
* Detect and de-identify sensitive data
* Audit LLM interactions with Logging

## Setup

Use the following GCP CloudShell tutorial. Follow the instructions to deploy the sample.

[![Open in Cloud Shell](https://gstatic.com/cloudssh/images/open-btn.svg)](https://ssh.cloud.google.com/cloudshell/open?cloudshell_git_repo=https://github.com/GoogleCloudPlatform/apigee-samples&cloudshell_git_branch=main&cloudshell_workspace=.&cloudshell_tutorial=llm-sse-security/docs/cloudshell-tutorial.md)

## Test Sample

### Install Dependencies

In [ ]:
!pip install -Uq google-genai
!pip install -Uq langchain==0.3.18
!pip install -Uq langchain-google-vertexai==2.0.12

### Authenticate your notebook environment (Colab only)
If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using Vertex AI Workbench.

In [8]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set the Variables

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
APIGEE_HOST="[your-apigee-host-domain]" # @param {type:"string"}
API_KEY="[your-apikey]" # @param {type:"string"}

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")
if not APIGEE_HOST or APIGEE_HOST == "[your-apigee-host-domain]":
    raise ValueError("Please set your APIGEE_HOST")
if not API_KEY or API_KEY == "[your-apikey]":
    raise ValueError("Please set your API_KEY")

API_ENDPOINT = "https://"+APIGEE_HOST+"/v1/samples/llm-sse-security"
LOCATION="us-east1"
MODEL="gemini-2.0-flash"

### Execute the Vertex AI Gemini model (Positive test cases)

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION,
    http_options=types.HttpOptions(api_version='v1', base_url=API_ENDPOINT, headers = {"x-apikey": API_KEY})
)

for chunk in client.models.generate_content_stream(
    model=MODEL, contents="Suggest a name of a flower shop"
):
    print(chunk.text, end='')

### Execute the Vertex AI Gemini model (Negative test cases)

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION,
    http_options=types.HttpOptions(api_version='v1', base_url=API_ENDPOINT, headers = {"x-apikey": API_KEY})
)

for chunk in client.models.generate_content_stream(
    model=MODEL, contents="Can you please check on my social security 123-45-6789?"
):
    print(chunk.text, end='')